In [1]:
#Actor Critic Method

import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from gym.wrappers.monitoring.video_recorder import VideoRecorder #For recording
import sys
sys.path.append('n:\\FFmpeg\\bin')

In [2]:
env = gym.make('CartPole-v0')

state = env.reset()
video_recorder = None
video_recorder = VideoRecorder(env, 'videos/start.mp4', enabled=True)
for _ in range(1000):
    env.render()
    video_recorder.capture_frame()
    env.step(env.action_space.sample()) # take a random action

print("Saved video.")
video_recorder.close()
video_recorder.enabled = False
env.close()

n:\python\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Saved video.


In [3]:
seed = 42
gamma = 0.99
max_steps_per_episode = 10000
env = gym.make('CartPole-v0')
env.seed(seed)
eps = np.finfo(np.float32).eps.item() #Machine limits for floating point types.
num_inputs = 4
num_actions = 2
num_hidden1 = 128
num_hidden2 = 256
learning_rate = 0.01

In [ ]:
inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden1,activation = 'relu')(inputs)
action = layers.Dense(num_actions, activation = 'softmax')(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs = inputs, outputs = [action,critic])

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = learning_rate)
huber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

while True:
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape :
        for timestep in range(1,max_steps_per_episode):
#             env.render()
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state,0)
            
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0,0])
            
            action = np.random.choice(num_actions, p = np.squeeze(action_probs))
            
            action_probs_history.append(tf.math.log(action_probs[0,action]))
            
            state, reward, done, _ = env.step(action)
            
            rewards_history.append(reward)
            episode_reward += reward
            if done :
                break
#         print("rewards",episode_reward)
        running_reward = (0.05* episode_reward) + ((1-0.05)* running_reward)
        returns = []
        discounted_sum = 0
        
        for i in rewards_history[::-1]:
            discounted_sum = i + (gamma* discounted_sum)
            returns.insert(0,discounted_sum)
            
        #Normalize
        
        returns = np.array(returns)
        returns = (returns - np.mean(returns))/(np.std(returns)+eps)
        returns = returns.tolist()
        
        history = zip(action_probs_history, critic_value_history, returns)
        
        actor_losses = []
        critic_losses = []
        
        for log_prob, value, ret in history:
            dif = ret - value
            actor_losses.append(-log_prob*dif)
            critic_losses.append(huber_loss(
            tf.expand_dims(value,0), tf.expand_dims(ret,0)))
            
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()
        
    episode_count += 1
    if episode_count %10 ==0:
        template = "running reward : {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))
        
    if running_reward > 195:
        print("Solved at episode {}!".format(episode_count))
        video_record = None
        video_record = VideoRecorder(env, 'videos/end.mp4', enabled=True)
        for _ in range(1000):
            env.render()
            video_record.capture_frame()
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state,0)
            action_probs, _ = model(state)
            action = np.random.choice(num_actions, p = np.squeeze(action_probs))
            state, reward, done, _ = env.step(action)
            
        print("Saved video.")
        video_record.close()
        video_record.enabled = False
        break


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
